In [1]:
import numpy as np
from matplotlib import pyplot as plt

import sys
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/')

import qml_interface as qmi

In [2]:
# load dE at 20 and 30 Ang for both molecules
dsdg_003664_lamc = np.loadtxt('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_003664/atomic_energies_with_mic.txt')
dsdg_003664_lamf = np.loadtxt('/home/misa/projects/Atomic-Energies/data/ueg_reference/number_lambda_points/dsgdb9nsd_003664_atomic_energies_fine_grid.txt')

dsdg_003700_lamc = np.loadtxt('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_003700/atomic_energies_with_mic.txt')
dsdg_003700_lamf = np.loadtxt('/home/misa/projects/Atomic-Energies/data/ueg_reference/number_lambda_points/dsgdb9nsd_003700_atomic_energies_fine_grid.txt')

In [3]:
# find the most similar atoms

# make coulomb representations
reps_dsdg_003664 = qmi.generate_atomic_representations([dsdg_003664_lamc], [len(dsdg_003664_lamc)])
reps_dsdg_003700 = qmi.generate_atomic_representations([dsdg_003700_lamc], [len(dsdg_003700_lamc)])

# calculate distance matrix

dmatrix = np.zeros((len(reps_dsdg_003664), len(reps_dsdg_003700)))
              
for i, r in enumerate(reps_dsdg_003664):
    for j, r2 in enumerate(reps_dsdg_003700):
        dmatrix[i,j] = np.linalg.norm(r-r2)

# find closest pairs
pairs = []

for i in range(dmatrix.shape[0]):
    dmatrix_row = dmatrix[i]
    ind = np.where( dmatrix_row == np.amin(dmatrix_row[np.where(dmatrix_row > 0)]))
    tmp = [i, ind[0][0]]
    #tmp.sort()
    pairs.append(tmp)
# pairs = []
# import itertools
# pairs.sort()
# pairs = list(pairs for pairs,_ in itertools.groupby(pairs))

In [4]:
# calculate the energy difference between these atoms for few and many lambda points
def delta_E(comp1, comp2, pairs):
    mu = []
    e = []
    e_at = []
    for p in pairs:
        mu.append(comp1[p[0],4] - comp2[p[1],4])
        e.append(comp1[p[0],5] - comp2[p[1],5])
        e_at.append(comp1[p[0],6] - comp2[p[1],6])
    return(np.array(mu), np.array(e), np.array(e_at))

In [5]:
# coarse grid (6 points)
muc, ec, e_atc = delta_E(dsdg_003664_lamc, dsdg_003700_lamc, pairs)
# fine grid
muf, ef, e_atf = delta_E(dsdg_003664_lamf, dsdg_003700_lamf, pairs)

In [16]:
ddmu = muc-muf

In [24]:
print('dd mu = ', (np.abs(ddmu)).mean(), '+/-', (np.abs(ddmu)).std())

dd mu =  0.003112474702578907 +/- 0.0033464975919379907


In [25]:
ddE_I = ec-ef

In [15]:
print('dd E_I = ', (np.abs(ddE_I)).mean(), '+/-', (np.abs(ddE_I)).std())

dd E_I =  0.010318384789749498 +/- 0.01100503828826174


In [18]:
ddE_at = e_atc - e_atf

In [19]:
print('dd E_I^at = ', (np.abs(ddE_at)).mean(), '+/-', (np.abs(ddE_at)).std())

dd E_I^at =  0.011422717639385942 +/- 0.008590362612555136


In [20]:
0.010318384789749498*627

6.469627263172935

In [22]:
dsdg_003700_lamc[:,0]

array([6., 6., 7., 7., 6., 6., 8., 1., 1., 1., 1., 1., 1.])

In [23]:
ddE_I

array([-1.58613302e-02, -2.86662668e-02, -5.91853650e-03, -3.79018657e-02,
        1.24372568e-02,  5.47859976e-03,  3.98291917e-03,  1.10360110e-04,
       -6.34543698e-05,  7.83135111e-04, -1.00383917e-02,  1.02229805e-02,
        2.67390550e-03])